In [2]:
import torch
import torchvision
from torch import nn
import lightning as L

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
print(f"Default device set to --> {device}")

Default device set to --> cuda


In [3]:
train_dataset = torchvision.datasets.MNIST(root="./data", download=True, train=True, transform=torchvision.transforms.ToTensor(), target_transform=None)
test_dataset = torchvision.datasets.MNIST(root="./data", download=True, train=False, transform=torchvision.transforms.ToTensor(), target_transform=None)

In [4]:
g = torch.Generator(device=device)
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, generator=g, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, generator=g, shuffle=False)

In [119]:
class BasicCNN(L.LightningModule):
    def __init__(self, n_channels):
        super(BasicCNN, self).__init__()

        self.act_fn = nn.ReLU()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=n_channels, out_channels=8*8*8, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32),
            nn.ReLU(),
            nn.Conv2d(in_channels=8*8*8, out_channels=8*8*8, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=8*8*8, out_channels=8*8*8, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32),
            nn.ReLU(),
            nn.Conv2d(in_channels=8*8*8, out_channels=8*8*8, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=0, out_features=10)
        )

        self.lr = 0.01
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x: torch.Tensor) -> torch.Tensor():
        print(x.shape)
        out1 = self.conv_block_1(x)
        out2 = self.conv_block_2(out1)
        #print(out2)
        print(out2.shape)

        out3 = self.classifier(out2)
        

        return logits

    def configure_optimizers(self):
        return torch.optim.AdamW(params=self.parameters(),lr=self.lr)

    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self.forward(x)
        loss = self.loss_fn(output, y)

        self.log("loss", loss)

        return loss

In [120]:

model = BasicCNN(n_channels=1)
x, y = train_dataset[0]

model(x.to(device))

torch.Size([1, 28, 28])
torch.Size([512, 7, 7])


/home/evie/torching/.venv/lib/python3.11/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x49 and 0x10)

In [118]:

trainer = L.Trainer(max_epochs=1,accelerator="gpu")
tuner = L.pytorch.tuner.Tuner(trainer=trainer)

find_lr = tuner.lr_find(model, train_dataloaders=train_dataloader, min_lr=0.0001, max_lr=1.0, early_stop_threshold=None)
new_lr = find_lr.suggestion()
model.lr = new_lr


trainer.fit(model=model, train_dataloaders=train_dataloader)
#out = model(next(iter(train_dataloader))[0])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/evie/torching/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


torch.Size([32, 1, 28, 28])
tensor([[[[4.8774e-03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 8.7106e-03],
          [3.5692e-03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.1812e-03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.3404e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [6.3756e-03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[2.5000e-02, 3.0357e-02, 3.9391e-02,  ..., 5.2493e-02,
           4.5937e-02, 2.1850e-02],
          [2.2222e-02, 3.8395e-02, 3.8125e-02,  ..., 3.7530e-02,
           5.1510e-02, 1.9921e-02],
          [2.1682e-02, 2.7438e-02, 2.0753e-02,  ..., 2.9587e-02,
           2.9156e-02, 2.1708e-02],
          ...,
          [4.0055e-02, 4.9812e-

ValueError: Expected input batch_size (1) to match target batch_size (32).

In [80]:
model.eval()
with torch.inference_mode():
    #x, y = next(iter(train_dataloader))
    x, y = train_dataset[987]

    logits = model(x)
    probs = torch.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1)

print(preds)
print(f"Predicted --> {train_dataset.classes[preds]}")
print(f"Expected --> {train_dataset.classes[y]}")

tensor([9])
Predicted --> 9 - nine
Expected --> 4 - four
